In [1]:
from pydantic import Field

from dslmodel import DSLModel, init_instant
from typing import List, Type
from typing import Optional

class Task(DSLModel):
    """
    Represents a task within a work session.
    """
    name: str = Field(..., description="The name of the task.")
    description: Optional[str] = Field(None, description="A brief description of the task.")
    duration_minutes: int = Field(..., description="Estimated duration to complete the task.")
    status: str = Field("Pending", description="Current status of the task (e.g., Pending, In Progress, Completed).")


class SMARTGoal(DSLModel):
    """
    Represents a SMART (Specific, Measurable, Achievable, Relevant, Time-bound) goal.
    """
    title: str = Field(..., description="The title of the SMART goal.")
    specific: str = Field(..., description="Specific details of the goal.")
    measurable: str = Field(..., description="Metrics to measure the goal's progress.")
    achievable: str = Field(..., description="Criteria that make the goal achievable.")
    relevant: str = Field(..., description="Relevance of the goal to broader objectives.")
    time_bound: str = Field(..., description="Time frame to achieve the goal.")


In [2]:
from dslmodel import from_prompt_chain

# Define the initial prompt
initial_prompt = "Grocery shopping for milk, cheese, and bread."

# Define the list of models in the desired sequence
models_chain = [SMARTGoal, Task]

init_instant()

# Execute the from_prompt_chain function
chain_results = from_prompt_chain(initial_prompt, models_chain)

# Access and print the results
for idx, result in enumerate(chain_results):
    if result:
        print(f"Result {idx + 1} ({result.__class__.__name__}): {result}")
    else:
        print(f"Result {idx + 1}: Failed to instantiate the model.")


INFO:dslmodel.utils.model_tools:Successfully instantiated SMARTGoal
INFO:dslmodel.utils.model_tools:Successfully instantiated Task


Result 1 (SMARTGoal): title='Grocery shopping for milk, cheese, and bread.' specific='Shopping for milk, cheese, and bread.' measurable='Number of items purchased (3)' achievable='Within a reasonable time frame (e.g., 1 hour)' relevant='To ensure household supplies are maintained.' time_bound='Within the next 24 hours.'
Result 2 (Task): name='Grocery shopping for milk, cheese, and bread.' description='Shopping for milk, cheese, and bread.' duration_minutes=3 status='Pending' title='Grocery shopping for milk, cheese, and bread.' specific='Shopping for milk, cheese, and bread.' measurable='Number of items purchased (3)' achievable='Within a reasonable time frame (e.g., 1 hour)' relevant='To ensure household supplies are maintained.' time_bound='Within the next 24 hours.'


In [3]:
from dslmodel import DSLModel, from_prompt_chain
from typing import List
from pydantic import Field

class Task(DSLModel):
    name: str = Field(..., description="Name of the task.")
    description: str = Field(..., description="Description of the task.")

class SequentialWorkflow(DSLModel):
    tasks: List[Task] = Field(..., description="List of tasks in sequence.")

# Define the models and initial prompt
initial_prompt = "Plan a product launch sequence."
models_chain = [Task, Task, Task]

# Use from_prompt_chain to create a sequence of tasks
workflow_result = from_prompt_chain(initial_prompt, models_chain)
print("Sequential Workflow:", workflow_result)


INFO:dslmodel.utils.model_tools:Successfully instantiated Task
INFO:dslmodel.utils.model_tools:Successfully instantiated Task
ERROR:dspy.primitives.assertions:2024-10-18T01:13:09.588054Z [error    ] AssertionError: You need to create a kwargs dict for Task
Validation error:
malformed node or string on line 1: <ast.Call object at 0x1264fbad0> [dspy.primitives.assertions] filename=assertions.py lineno=88
ERROR:dslmodel.dspy_modules.gen_pydantic_instance:Error during validation: You need to create a kwargs dict for Task
Validation error:
malformed node or string on line 1: <ast.Call object at 0x1264fbad0>
Output:
Task(name='Product Launch Sequence', description='A planned sequence of events for the product launch.')
18:13:09 - LiteLLM:INFO: utils.py:3052 - 
LiteLLM completion() model= llama-3.1-8b-instant; provider = groq
INFO:LiteLLM:
LiteLLM completion() model= llama-3.1-8b-instant; provider = groq
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 2

Sequential Workflow: [Task(name='Product Launch Sequence', description='A planned sequence of events for the product launch.'), Task(name='Product Launch Sequence', description='A planned sequence of events for the product launch.'), Task(name='Product Launch Sequence', description='A planned sequence of events for the product launch.')]


In [4]:
from dslmodel import from_prompt_chain, init_instant, init_text, DSLModel, Field

init_text()

class InitialDraft(DSLModel):
    draft: str = Field(..., description="The initial draft content.")

class Feedback(DSLModel):
    suggestions: str = Field(..., description="Feedback for improving the draft.")

class RefinedDraft(DSLModel):
    improved_draft: str = Field(..., description="The refined version of the draft.")

class IterationCycle(DSLModel):
    initial: InitialDraft = Field(..., description="Initial draft of the document.")
    feedback: Feedback = Field(..., description="Feedback on the initial draft.")
    refined: RefinedDraft = Field(..., description="Refined draft after feedback.")

initial_prompt = "Write a draft for a blog post on AI ethics."
models_chain = [InitialDraft, Feedback, RefinedDraft]

# Use from_prompt_chain to refine a draft iteratively
iteration_result = from_prompt_chain(initial_prompt, models_chain)
print("Iteration Cycle:", iteration_result)


2024-10-17 18:48:03.875 | DEBUG    | dslmodel.utils.log_tools:log_debug:88 - Using prompt for InitialDraft: Write a draft for a blog post on AI ethics.
2024-10-17 18:48:04.489 | INFO     | dslmodel.utils.log_tools:log_info:83 - Generated InitialDraft: draft='The ethics of artificial intelligence is a rapidly evolving field that has garnered significant attention in recent years. As AI systems become increasingly integrated into various aspects of our lives, it is essential to consider the moral implications of their development and deployment. This blog post aims to explore the key issues surrounding AI ethics, including transparency, accountability, and fairness.'
2024-10-17 18:48:04.491 | ERROR    | dslmodel.utils.log_tools:log_error:98 - Failed to generate InitialDraft with prompt 'Write a draft for a blog post on AI ethics.': 'InitialDraft' object has no attribute 'analysis'
2024-10-17 18:48:04.492 | DEBUG    | dslmodel.utils.log_tools:log_debug:88 - Using prompt for Feedback: Writ

Iteration Cycle: [InitialDraft(draft='The ethics of artificial intelligence is a rapidly evolving field that has garnered significant attention in recent years. As AI systems become increasingly integrated into various aspects of our lives, it is essential to consider the moral implications of their development and deployment. This blog post aims to explore the key issues surrounding AI ethics, including transparency, accountability, and fairness.'), None, Feedback(suggestions='Consider discussing the potential biases in AI decision-making, the importance of transparency in AI development, and the need for accountability in AI applications.'), None, RefinedDraft(improved_draft='The rapid development of artificial intelligence has raised significant concerns about its impact on society. As AI becomes increasingly integrated into our daily lives, it is essential to consider the ethical implications of its use. This blog post will explore the key issues surrounding AI ethics, including bi

In [6]:
class Goal(DSLModel):
    title: str = Field(..., description="Title of the goal.")
    specific: str = Field(..., description="Specifics of the goal.")

class Step(DSLModel):
    action: str = Field(..., description="Action to achieve the goal.")

class Resource(DSLModel):
    name: str = Field(..., description="Resource needed for the step.")

class GoalPlanning(DSLModel):
    goal: Goal = Field(..., description="The overarching goal.")
    steps: list[Step] = Field(..., description="Steps to achieve the goal.")
    resources: list[Resource] = Field(..., description="Resources required.")

initial_prompt = "Plan for launching a new product in the market."
models_chain = [Goal, Step, Resource]

# Generate goal, steps, and resources using from_prompt_chain
planning_result = from_prompt_chain(initial_prompt, models_chain)
print("Goal Planning:", planning_result)


2024-10-17 18:48:41.583 | DEBUG    | dslmodel.utils.log_tools:log_debug:88 - Using prompt for Goal: Plan for launching a new product in the market.
2024-10-17 18:48:42.574 | INFO     | dslmodel.utils.log_tools:log_info:83 - Generated Goal: title='New Product Launch' specific='Plan for launching a new product in the market.'
2024-10-17 18:48:42.575 | ERROR    | dslmodel.utils.log_tools:log_error:98 - Failed to generate Goal with prompt 'Plan for launching a new product in the market.': 'Goal' object has no attribute 'analysis'
2024-10-17 18:48:42.576 | DEBUG    | dslmodel.utils.log_tools:log_debug:88 - Using prompt for Step: Plan for launching a new product in the market.
2024-10-17 18:48:42.909 | INFO     | dslmodel.utils.log_tools:log_info:83 - Generated Step: action='Plan for launching a new product in the market.'
2024-10-17 18:48:42.910 | ERROR    | dslmodel.utils.log_tools:log_error:98 - Failed to generate Step with prompt 'Plan for launching a new product in the market.': 'Step' 

Goal Planning: [Goal(title='New Product Launch', specific='Plan for launching a new product in the market.'), None, Step(action='Plan for launching a new product in the market.'), None, Resource(name='Plan for launching a new product in the market'), None]
